<a href="https://colab.research.google.com/github/Yi-Wei-Lin/Tibame_ML_20210602/blob/main/Cabbage_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
# train_df = pd.read_csv("banana.csv", encoding="utf-8")
train_df = pd.read_excel("Cabbage_v2.xlsx")

# Date 民國 to 西元
def date_df(d):
  d = d.split("/")
  year = str(int(d[0])+1911)
  month = (d[1])
  day = (d[2])
  d = (f"{year}-{month}-{day}")
  return d
date_df = pd.DataFrame({"Date":train_df["Date"].apply(date_df)})
train_df.update(date_df)

# Month method
def Month(m):
  Month_data = int(m.split("-")[1])
  return Month_data
train_df["Month"] = train_df["Date"].apply(Month)

# Week_day method
def week_day(d):
  d = d.split("-")
  year = int(d[0])
  month = int(d[1])
  day = int(d[2])
  temp = pd.Timestamp(f"{year}-{month}-{day}")
  week_day = temp.dayofweek + 1 #temp.dayofweek 0(禮拜一) 開始, +1是因為比較直接禮拜幾
  return week_day
train_df["Week_day"] = train_df["Date"].apply(week_day)
train_df["Year"] = train_df["Date"].apply(lambda d:int(d.split("-")[0]))

# Rest_market
market_df = pd.DataFrame({"Market":train_df["Market"].apply(lambda t:t.split(" ")[1])})
train_df.update(market_df)

# Volume chnage from str to int
# train_df["Volume"] = train_df["Volume"].apply(lambda x:int(x.replace(",","")))

# Up_price change from onj to float
# train_df["Up_price"] = train_df["Up_price"].apply(lambda x:float(x))
market_df["Market"]
train_df

,Date,Market,Product,Up_price,Mid_price,Low_price,Avg_price,Avg_price_diff,Volume,Volume_diff,Month,Week_day,Year
0,2001-01-01,台北二,LA1 甘藍 初秋,10.4,7.8,5.7,7.9,NaN,28434.0,0.0,1,1,2001
1,2001-01-02,台北二,LA1 甘藍 初秋,13.0,11.2,7.5,10.8,37.0,20183.0,-29.0,1,2,2001
2,2001-01-03,台北二,LA1 甘藍 初秋,11.0,8.9,6.9,8.9,-18.0,43627.0,116.0,1,3,2001
3,2001-01-05,台北二,LA1 甘藍 初秋,8.7,6.9,5.1,6.9,NaN,15317.0,NaN,1,5,2001
4,2001-01-06,台北二,LA1 甘藍 初秋,5.8,4.2,3.0,4.3,-38.0,50395.0,229.0,1,6,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86332,2021-06-01,花蓮市,LA1 甘藍 初秋,32.0,32.0,30.0,31.6,37.0,1041.0,649.0,6,2,2021
86333,2021-06-02,花蓮市,LA1 甘藍 初秋,36.0,35.3,32.9,35.0,11.0,810.0,-22.0,6,3,2021
86334,2021-06-03,花蓮市,LA1 甘藍 初秋,34.4,34.0,33.8,34.1,-3.0,768.0,-5.0,6,4,2021
86335,2021-06-04,花蓮市,LA1 甘藍 初秋,16.0,16.0,16.0,16.0,-53.0,54.0,-93.0,6,5,2021


In [4]:
train_df["Date"] = pd.to_datetime(train_df["Date"])
train_df["Date"]

0       2001-01-01
1       2001-01-02
2       2001-01-03
3       2001-01-05
4       2001-01-06
           ...    
86332   2021-06-01
86333   2021-06-02
86334   2021-06-03
86335   2021-06-04
86336   2021-06-08
Name: Date, Length: 86337, dtype: datetime64[ns]

In [5]:
train_df
train_df.dtypes

Date              datetime64[ns]
Market                    object
Product                   object
Up_price                 float64
Mid_price                float64
Low_price                float64
Avg_price                float64
Avg_price_diff           float64
Volume                   float64
Volume_diff              float64
Month                      int64
Week_day                   int64
Year                       int64
dtype: object

In [6]:
# 填補市場休市欄位
market_list = train_df["Market"].value_counts().index.tolist()
m_total = []
for i,mark_name in enumerate(market_list):
  a = train_df["Market"] == mark_name
  start_time = train_df[a].reset_index(drop=True)["Date"][0]
  t_index = pd.date_range(start_time, '2021-06-19', freq='D') #start_time by 市場別
  T_df = pd.DataFrame(t_index, columns=['Date'])
  m = train_df[a].merge(T_df, how="right",sort=True)
  most = m.value_counts().idxmax()
  m["Market"]=m["Market"].fillna(most[1])
  m["Product"]=m["Product"].fillna(most[2])
  na = m["Volume"].isna()
  m["Month"] = m["Date"].apply(lambda x:int(str(x).split("-")[1]))
  m["Week_day"] = m["Date"].apply(lambda x:pd.Timestamp(str(x).split()[0]).dayofweek+1)
  m["Year"] = m["Date"].apply(lambda x:int(str(x).split("-")[0]))
  import math
  def restday(s):
    if s > 0:
      return 0
    else:
      return 1
  m["Rest_day"] = m["Volume"].apply(restday) #判斷 restday 0 or 1
  Avg_price_diff = []
  Volume_diff = []
  Avg_price_diff.append(m["Avg_price_diff"][0])
  Volume_diff.append(m["Volume_diff"][0])
  for i in range(len(m)-1):
    Avg_price_diff_value = (m["Avg_price"][i+1] - m["Avg_price"][i]) / m["Avg_price"][i]
    Avg_price_diff_value = round(Avg_price_diff_value*100, 2)
    Avg_price_diff.append(Avg_price_diff_value)

    Volume_diff_value = (m["Volume"][i+1] - m["Volume"][i]) / m["Volume"][i]
    Volume_diff_value = round(Volume_diff_value*100, 2)
    Volume_diff.append(Volume_diff_value)
  m["Avg_price_diff"] = Avg_price_diff
  m["Volume_diff"] = Volume_diff
  globals()[f"m{i}"] = m
  m_total.append(globals()[f"m{i}"])
print(m.dtypes)
len(market_list)
final_data = pd.concat(m_total).reset_index(drop=True)

Date              datetime64[ns]
Market                    object
Product                   object
Up_price                 float64
Mid_price                float64
Low_price                float64
Avg_price                float64
Avg_price_diff           float64
Volume                   float64
Volume_diff              float64
Month                      int64
Week_day                   int64
Year                       int64
Rest_day                   int64
dtype: object


In [8]:
# final_data.to_csv("Cabbage_v2.csv", encoding="utf-8", index=False)
final_data.to_excel("Cabbage_v3.xlsx", encoding="utf-8", index=False)
final_data

,Date,Market,Product,Up_price,Mid_price,Low_price,Avg_price,Avg_price_diff,Volume,Volume_diff,Month,Week_day,Year,Rest_day
0,2001-01-01,西螺鎮,LA1 甘藍 初秋,11.4,8.8,6.3,8.8,-3.00,216770.0,-2.00,1,1,2001,0
1,2001-01-02,西螺鎮,LA1 甘藍 初秋,10.4,8.0,5.0,7.9,-10.23,220460.0,1.70,1,2,2001,0
2,2001-01-03,西螺鎮,LA1 甘藍 初秋,8.4,6.3,4.3,6.3,-20.25,222380.0,0.87,1,3,2001,0
3,2001-01-04,西螺鎮,LA1 甘藍 初秋,7.4,5.8,4.0,5.8,-7.94,222960.0,0.26,1,4,2001,0
4,2001-01-05,西螺鎮,LA1 甘藍 初秋,7.0,5.3,4.0,5.4,-6.90,224890.0,0.87,1,5,2001,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106780,2021-06-15,板橋區,LA1 甘藍 初秋,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,2,2021,1
106781,2021-06-16,板橋區,LA1 甘藍 初秋,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,3,2021,1
106782,2021-06-17,板橋區,LA1 甘藍 初秋,21.0,13.8,8.0,14.1,NaN,36960.5,NaN,6,4,2021,0
106783,2021-06-18,板橋區,LA1 甘藍 初秋,20.1,12.2,7.0,12.8,-9.22,27821.8,-24.73,6,5,2021,0
